検証用・雑記用のnotebook

# モジュールのインポート

In [2]:
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm

from modules.constants import LocalDirs
from modules import preparing
from modules import preprocessing
from modules import training
from modules import simulation
from modules import policies
%load_ext autoreload

In [3]:
pd.read_pickle('data/raw/return_tables/return_tables_2021.pickle')

,0,1,2,3
202106030610,単勝,8,490,2
202106030610,複勝,8br3br6,190br170br170,4br2br3
202106030610,枠連,3 - 7,1090,5
202106030610,馬連,3 - 8,1330,3
202106030610,ワイド,3 - 8br6 - 8br3 - 6,510br400br660,3br1br11
...,...,...,...,...
202106030609,馬連,3 - 4,540,2
202106030609,ワイド,3 - 4br1 - 4br1 - 3,200br170br230,2br1br3
202106030609,馬単,4 → 3,730,2
202106030609,三連複,1 - 3 - 4,470,1


In [6]:
pd.read_csv('data/master/horse_id.csv')

,horse_id,encoded_id
0,2018106541,0
1,2018103708,1
2,2018103805,2
3,2018101171,3
4,2018103832,4
...,...,...
11516,2019101360,11516
11517,2019101500,11517
11518,2019104312,11518
11519,2018102219,11519


In [4]:
max(pd.read_csv('data/master/horse_id.csv')['encoded_id'])

11520